In [3]:
# Instala o YOLOv8 (ultralytics) e outras bibliotecas necessárias
!pip install --upgrade --quiet ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import io
import cv2
from base64 import b64decode, b64encode
from PIL import Image
from ultralytics import YOLO
from IPython.display import display, Javascript
from google.colab.output import eval_js

# --- CONFIGURAÇÃO ---
MODELO_WEB_PATH = 'best.pt' # Certifique-se de que o arquivo está no Colab
MODELO = YOLO(MODELO_WEB_PATH)
IMG_SHAPE = [640, 640]

def js_to_image(js_response):
    """Converte a captura do navegador para OpenCV."""
    _, b64_str = js_response['img'].split(',')
    jpeg_bytes = b64decode(b64_str)
    image_pil = Image.open(io.BytesIO(jpeg_bytes))
    return cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)

def video_stream():
    """O 'Pilar' JavaScript para abrir a webcam no Colab."""
    js = Javascript(f'''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var shutdown = false;

    async function createDom() {{
        if (div !== null) return stream;
        div = document.createElement('div');
        div.style.border = '2px solid black';
        div.style.width = '600px';
        document.body.appendChild(div);

        video = document.createElement('video');
        video.style.display = 'block';
        video.width = 600;
        video.setAttribute('playsinline', '');
        video.onclick = () => {{ shutdown = true; }};
        stream = await navigator.mediaDevices.getUserMedia({{video: true}});
        div.appendChild(video);

        imgElement = document.createElement('img');
        imgElement.style.position = 'absolute';
        imgElement.style.zIndex = 1;
        div.appendChild(imgElement);

        video.srcObject = stream;
        await video.play();

        captureCanvas = document.createElement('canvas');
        captureCanvas.width = {IMG_SHAPE[0]};
        captureCanvas.height = {IMG_SHAPE[1]};
        return stream;
    }}

    async function takePhoto(imgData) {{
        if (shutdown) {{
            if (stream) stream.getVideoTracks()[0].stop();
            div.remove();
            return '';
        }}
        await createDom();
        if (imgData != "") {{
            var rect = video.getBoundingClientRect();
            imgElement.style.top = rect.top + "px";
            imgElement.style.left = rect.left + "px";
            imgElement.src = imgData;
        }}
        captureCanvas.getContext('2d').drawImage(video, 0, 0, {IMG_SHAPE[0]}, {IMG_SHAPE[1]});
        return {{ 'img': captureCanvas.toDataURL('image/jpeg', 0.8) }};
    }}
    window.takePhoto = takePhoto;
    ''')
    display(js)

# --- EXECUÇÃO ---
video_stream()
img_data = ''

while True:
    js_response = eval_js(f'takePhoto("{img_data}")')
    if not js_response: break

    frame = js_to_image(js_response)

    # Inferência (conf=0.4 para filtrar ruído)
    results = MODELO(frame, imgsz=640, verbose=False, conf=0.4)[0]

    # Desenha as caixas
    annotated_frame = results.plot()

    # Prepara o "overlay" para enviar de volta ao navegador
    _, buffer = cv2.imencode('.png', annotated_frame)
    img_base64 = b64encode(buffer).decode('utf-8')
    img_data = f'data:image/png;base64,{img_base64}'
